### ***분석 목적***
- **월 평균 리뷰 수 산출** : 브랜드별 고객 반응 및 활동성 비교
- **운영 기간 보정** : 단순 누적 리뷰 수가 아닌, 동일 기간 기준 평균 지표를 사용하여 비교
- **기간 차이에 따른 왜곡 제거** : 지점별·브랜드별 리뷰 생성 속도를 공정하게 비교

### ***데이터 수집 방법***
- ***리뷰 수집 방식***
    - 네이버 플레이스 리뷰 최신순 크롤링
    - 지점별 리뷰 데이터를 개별 수집
- ***기간 필터링***
    - 기준 시작일: 2025-01-01
    - 기준 종료일: 분석 시점 기준 최근 1년
    - 분석 대상:
        - 기준일 이후 작성된 리뷰만 사용
        - 기준일 이전 리뷰는 분석에서 제외
- ***사용 데이터***
    - 지점별 리뷰 데이터
        - place_id
        - 지점명
        - 리뷰 작성일
        - 리뷰 유형(방문자 리뷰/블로그 리뷰 등)
        - 리뷰수

### ***평균 리뷰 수 계산***
- ***월 평균 리뷰 수*** = 해당 기간 내 누적 리뷰 수 ÷ 기간(개월)
- ***일 평균 리뷰 수*** = 해당 기간 내 누적 리뷰 수 ÷ 기간(일)
- 3년간 리뷰 1000개 → 약 36개월 → 월평균 ≈ 27.8개
- 1개월간 리뷰 200개 → 월평균 = 200개
- ***기간 차이를 반드시 고려하여 브랜드/지점 비교***
### ==============================
- 1년간 리뷰 120개 → 12개월 → 월평균 = 10개
- 6개월간 리뷰 60개 → 월평균 = 10개

### ***분석 대상***
- 브랜드별 비교 : 역전할머니, 금별맥주
- 분석 단위 : 지점별 분석, 브랜드별 집계 분석
- 리뷰 유형별 구분 가능: 블로그 리뷰, 방문자 리뷰 

### ***분석 흐름***
- CSV 불러오기
    - 역전할머니, 금별맥주 리뷰 데이터 불러오기 (place_id, 지점명, 리뷰수, 리뷰날짜)
- 기간 필터링
    - 2025-01-01 이후 최근 1년 리뷰만 선택
- 지점별 누적 리뷰 수 계산
    - 필터링된 리뷰를 기준으로 지점별 누적 수 산출
- 월평균 리뷰 계산
    - 누적 리뷰 수 ÷ 12개월 → 월평균 리뷰
    - 누적 리뷰 수 ÷ 365일 → 일평균 리뷰
- CSV 저장 및 결과 확인
    - 브랜드별 월평균 리뷰 수 확인 가능

### ***평균 사용한 이유***
- 시군구 내 개별 매장의 편차를 완화하고, 상권 전체의 평균적인 수요 수준을 대표하기 위해 평균값을 사용

### ***최근 데이터만 사용한 이유***
- 출점 전략 분석 목적상 과거보다 최근 수요를 반영하는 것이 더 적절하다고 판단

In [1]:
import pandas as pd
from datetime import datetime, timedelta

In [5]:
# 1. 파일 경로
# =====================================
BASE_PATH = r"C:\Users\user\Desktop\데이터분석\출력"

FILES = {
    "역전할머니": f"{BASE_PATH}\\역전_리뷰수.csv",
    "금별맥주": f"{BASE_PATH}\\금별_리뷰수.csv"
}

OUTPUT_CSV = f"{BASE_PATH}\\월평균_리뷰_최근1년.csv"

In [3]:
# 2. 분석 기간 설정
# =====================================
END_DATE = datetime.today()
START_DATE = END_DATE - timedelta(days=365)

MONTHS = 12
DAYS = 365

In [7]:
# 3. 브랜드별 계산 (리뷰수 CSV 기준)
# =====================================
result_frames = []

for brand, path in FILES.items():
    df = pd.read_csv(path)

    # 컬럼 검증
    if "리뷰수" not in df.columns:
        raise ValueError(f"[{brand}] 리뷰수 컬럼이 없습니다: {df.columns}")

    # 지점별 누적 리뷰 수 (이미 집계된 값 사용)
    df_grouped = df[["place_id", "지점명", "리뷰수"]].copy()
    df_grouped.rename(columns={"리뷰수": "누적_리뷰수"}, inplace=True)

    # 월평균 / 일평균 계산
    df_grouped["월평균_리뷰수"] = df_grouped["누적_리뷰수"] / 12
    df_grouped["일평균_리뷰수"] = df_grouped["누적_리뷰수"] / 365

    # 메타 정보
    df_grouped["브랜드"] = brand
    df_grouped["분석기준"] = "최근 1년 누적 리뷰수 기준"
    df_grouped["분석개월수"] = 12

    result_frames.append(df_grouped)

In [8]:
# 4. 통합 데이터프레임 생성
# =====================================
final_df = pd.concat(result_frames, ignore_index=True)

In [9]:
# 5. CSV 저장
# =====================================
final_df.to_csv(OUTPUT_CSV, index=False, encoding="utf-8-sig")

print(f"[완료] 월평균 리뷰 계산 결과 저장 → {OUTPUT_CSV}")
print(final_df.head())

[완료] 월평균 리뷰 계산 결과 저장 → C:\Users\user\Desktop\데이터분석\출력\월평균_리뷰_최근1년.csv
     place_id     지점명  누적_리뷰수    월평균_리뷰수   일평균_리뷰수    브랜드             분석기준  \
0  1868359886     구청점     283  23.583333  0.775342  역전할머니  최근 1년 누적 리뷰수 기준   
1  1323812622    명지대점     182  15.166667  0.498630  역전할머니  최근 1년 누적 리뷰수 기준   
2  1311695977  서울신풍역점      10   0.833333  0.027397  역전할머니  최근 1년 누적 리뷰수 기준   
3  1650551571   서울상봉점     431  35.916667  1.180822  역전할머니  최근 1년 누적 리뷰수 기준   
4  1404198758     오류점       1   0.083333  0.002740  역전할머니  최근 1년 누적 리뷰수 기준   

   분석개월수  
0     12  
1     12  
2     12  
3     12  
4     12  


월간 검색량과 블로그 누적 발행량은 2025년 1월 1일 이후 데이터를 기준으로 수집되었으며, 시군구 단위 분석을 위해 동일 시군구 내 매장들의 값을 평균하여 산출하였다. 결측치는 수요 또는 관심이 발생하지 않은 것으로 간주하여 0으로 처리하였다